In [ ]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(5) 
    return soup

In [ ]:
def get_data_indiatoday_web(soup):
    
    articles = soup.find_all("div",attrs={"class":"catagory-listing"})
    
    # Declaration
    articles_topic = []
    articles_content = []
    for article in articles:
        # Get article's topic
        if article.find("p") != None and article.find("h2") != None:
            topic = article.find("h2").get_text()
            articles_topic.append(topic)

            # Get article's content - there are lots of videos description so we take the article's summary
            articles_content.append(article.find("p").get_text()) 
        
    return articles_topic, articles_content

In [ ]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'indiatoday Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["business", "world", "technology", "science", "health"]

print("Start scraping from indiatoday!")

def scraping_with_selenium_from_url(url):
        driver = webdriver.Chrome()
        driver.get(url)
        driver.maximize_window()
        time.sleep(10)
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        topic, content = get_data_indiatoday_web(soup)
        insert_data_to_csv(topic, category, content)
        next_button = driver.find_element(by=By.LINK_TEXT, value=('Next')).click()
        for i in range (1, 15):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(10)
            html = driver.page_source
            soup = BeautifulSoup(html,"html.parser")
            topic, content = get_data_indiatoday_web(soup)
            insert_data_to_csv(topic, category, content)
            next_button = driver.find_element(by=By.LINK_TEXT, value=('Next')).click()
        driver.close()
  
for category in categories:
    if category == "technology":
        scraping_with_selenium_from_url(f"https://www.indiatoday.in/{category}/news")
    else:
        scraping_with_selenium_from_url(f"https://www.indiatoday.in/{category}")
        
print("Finish scraping from indiatoday.in!")
     